In [ ]:
import xarray as xr
import py12box
from pyprojroot import here
import pandas as pd
import utils
from scipy.optimize import curve_fit
import numpy as np

Write file of monthly baseline obs:

In [ ]:
utils.obs_write("CFC-11", here() / "data/example", "CFC-11")

Read file:

In [ ]:
df = utils.obs_read("CFC-11", here() / "data/example", "CFC-11")
df

Plot mole fraction columns

In [ ]:
idx = pd.IndexSlice
df.loc[:, idx[:, :, "mf"]].plot()

In [ ]:
sites = set([c[0] for c in df.columns])

In [ ]:
len(wh_finite)

In [ ]:
import matplotlib.pyplot as plt

def f(x, a, b, c):
    return a + b*x + c*x**2

x = utils.decimal_date(df.index)
wh_first = np.where(x < x[0] + 10.)
x_first = x[wh_first]

global_mean_y1 = np.zeros(12)
global_mean_count = 0

for s in sites:
    y = np.squeeze(df.loc[:, idx[s, :, "mf"]].values)

    plt.plot(x, y, '.')
    
    y_first = y[wh_first]
    
    wh_finite = np.where(np.isfinite(y_first))
    
    if len(wh_finite[0]) > 0:
        if wh_finite[0][0] < 24:
            x_finite = x_first[wh_finite]
            y_finite = y_first[wh_finite]

            fit_params, pcov = curve_fit(f, x_finite, y_finite)
            
            global_mean_y1 += f(x_first[0:12], *fit_params)
            global_mean_count += 1
            
            plt.plot(x_first, f(x_first, *fit_params))
    
global_mean_y1 /= global_mean_count

plt.plot(x_first[0:12], global_mean_y1, linewidth = 2.)

global_ic = global_mean_y1[0]
global_ic_trend = global_mean_y1[11] - global_mean_y1[0]

lifetime = 50.
Matm = 5.1170001e+21 / 28.97
mol_mass = 137.

q0 = Matm * (global_ic_trend + global_ic / lifetime) / 1e12 * mol_mass / 1e12

In [ ]:
df.loc[:, idx["MHD", :, "mf"]]